In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import pywt
import matplotlib.pyplot as plt

from mne import read_epochs, set_log_level, compute_rank, concatenate_epochs

from scipy.stats import kurtosis, skew, moment, entropy, norm
from mne.decoding import CSP, Scaler
from mrmr import mrmr_classif
from ReliefF import ReliefF
from sklearn import svm
from random import randint
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from pyriemann.tangentspace import TangentSpace
from pyriemann.estimation import Covariances, Kernels
from sklearn.model_selection import train_test_split, LeaveOneOut, StratifiedShuffleSplit, StratifiedKFold, cross_val_score, GridSearchCV,ShuffleSplit
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from jupyterthemes.stylefx import set_nb_theme
set_nb_theme('gruvboxd')

In [2]:
set_log_level('warning')
epochs = read_epochs('ica_epo.fif').pick('eeg').filter(0,240)
epochs.drop_channels(epochs.info['bads'])
epochs.apply_baseline((-1.4,-0.4))

Number of events,120
Events,left: 24r_pinch: 25r_stop: 25rest: 21right: 25
Time range,-2.000 – 7.999 sec
Baseline,-1.400 – -0.400 sec


In [3]:
lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
nuSvm = svm.NuSVC(gamma="auto")
knn = KNeighborsClassifier(n_neighbors=3)
relief = ReliefF(n_neighbors=3, n_features_to_keep=10)
linear_svm = svm.SVR(kernel="linear")
svm_rbf = svm.SVC(kernel="rbf")
rf = RandomForestClassifier(random_state=0)
le = LabelEncoder()
tangent_space = TangentSpace()
scaler = Scaler(info=epochs.info)

In [4]:
def dwt_det_coeff(x, db='db2'):
    aprx, det = pywt.dwt(x,db)
    return det

def dwt_aprox_coeff(x, db='db2'):
    aprx, det = pywt.dwt(x,db)
    return aprx

def rms(x):
    return np.sqrt(np.mean(x**2))

def slope(x):
    t = np.linspace(0, len(x)-1, len(x))
    return np.polyfit(t, x, 1)[0]

def autocorr(x):
    return float(np.correlate(x,x))
    
def temp_centroid(x):
    nom = [x[i] * i for i in range(x.shape[0])]
    return sum(nom)/sum(x)

def energy(x):
    return sum(x**2)

def med_abs_diff(x):
    return np.median(np.abs(np.diff(x)))

def mean_abs_diff(x):
    return np.mean(np.abs(np.diff(x)))

def calc_centroid(x, fs=1024):
    energy = np.array(x) ** 2
    t = range(len(x))
    t = [float(x) / fs for x in t]
    t_energy = np.dot(np.array(t), np.array(energy))
    energy_sum = np.sum(energy)

    if energy_sum == 0 or t_energy == 0:
        centroid = 0
    else:
        centroid = t_energy / energy_sum

    return centroid


left vs rest

In [5]:
conditions = ['left','rest']
subset = epochs[conditions].copy()
subset.drop_channels(subset.info['bads'])
subset = subset.pick(['eeg'])
subset = subset.apply_baseline((-1.4,-0.4))
y = le.fit_transform(subset.events[:,2])
train_data = subset.copy().crop(0.4,1.2).get_data()    
time_config = (3,0.4,300,100)

In [6]:
y = le.fit_transform(subset.events[:,2])
chance = np.mean(y == y[0])
chance = max(chance, 1. - chance)

In [32]:
print(genConf(train_data.shape[0], 0.01))
print(genConf(train_data.shape[0], 0.05))

71.61025808329968
67.17394012712947


In [9]:
test_dwt('db26', 7)

0.7200000000000001   0.85  pca
0.7266666666666667   0.87  pca
0.7266666666666668   0.89  pca
0.74   0.91  pca
0.7533333333333333   0.93  pca


In [10]:
pca = PCA(n_components=.93)

In [ ]:
test_levels('db26')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.40it/s]


0.9200000000000002 with  1 and  2


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  2.51it/s]


In [16]:
test_levels('db26')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.01it/s]


0.8222222222222223 with  3 and  4


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.31it/s]


0.888888888888889 with  3 and  5


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.67it/s]


In [12]:
test_extra_lvl('db26', 6, 7, 'lda')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.35it/s]


pca 0.5466666666666666
rfe 0.72
mrmr 0.9066666666666666
rf 0.5199999999999999


In [13]:
test_extra_lvl('db26', 3, 5, 'lda')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.65it/s]


pca 0.5266666666666667
rfe 0.7666666666666666
mrmr 0.8466666666666667
rf 0.4933333333333333


In [14]:
test_extra_lvl('db26', 1, 2, 'lda')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.71it/s]


pca 0.4600000000000001
rfe 0.8266666666666668
mrmr 0.9266666666666667
rf 0.43999999999999995


left vs rest

In [15]:
conditions = ['left','right']
subset = epochs[conditions].copy()
subset.drop_channels(subset.info['bads'])
subset = subset.pick(['eeg'])
subset = subset.apply_baseline((-1.4,-0.4))
y = le.fit_transform(subset.events[:,2])
train_data = subset.copy().crop(0.2,1).get_data()    
time_config = (3,0.4,300,100)

In [16]:
y = le.fit_transform(subset.events[:,2])
chance = np.mean(y == y[0])
chance = max(chance, 1. - chance)

In [17]:
print(genConf(train_data.shape[0], 0.01))
print(genConf(train_data.shape[0], 0.05))

68.79890857530535
64.52940869445021


In [11]:
test_dwt('db9', 4)

0.5356617647058823   0.85  pca
0.5636029411764707   0.87  pca
0.5970588235294118   0.89  pca
0.6275735294117647   0.91  pca


In [15]:
test_levels('db9')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.95it/s]


0.875 with  1 and  2


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.11it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.30it/s]


In [20]:
test_levels('db9')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.70it/s]


0.7352941176470589 with  3 and  4


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.81it/s]


0.7941176470588235 with  3 and  5


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.96it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.76it/s]


0.8394607843137255 with  4 and  5


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.05it/s]


0.8578431372549019 with  4 and  6


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.45it/s]


In [18]:
test_extra_lvl('db9', 2, 4, 'lda')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.28it/s]


pca 0.5775735294117647
rfe 0.6948529411764707
mrmr 0.80625
rf 0.5444852941176471


In [19]:
test_extra_lvl('db9', 1, 2, 'lda')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.31it/s]


pca 0.5643382352941175
rfe 0.6889705882352941
mrmr 0.85
rf 0.4875


In [20]:
test_extra_lvl('db9', 4, 6, 'lda')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.84it/s]


pca 0.5242647058823529
rfe 0.65
mrmr 0.775
rf 0.48492647058823535


In [8]:
conditions = ['left','right','rest']
subset = epochs[conditions].copy()
subset.drop_channels(subset.info['bads'])
subset = subset.pick(['eeg'])
subset = subset.apply_baseline((-1.4,-0.4))
y = le.fit_transform(subset.events[:,2])
train_data = subset.copy().crop(0.2,1).get_data()    
time_config = (3,0.4,300,100)
y = le.fit_transform(subset.events[:,2])
chance = np.max([np.mean(y == y[0]), np.mean(y == y[1]), np.mean(y == y[2])])
print(genConf(train_data.shape[0], 0.01))
print(genConf(train_data.shape[0], 0.05))

51.01087740595296
47.54071191236497


In [12]:
test_levels('db9')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.11it/s]


0.6260869565217391 with  1 and  2


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.97it/s]


0.682608695652174 with  1 and  5


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.61it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.30it/s]


0.6956521739130435 with  2 and  3


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.21it/s]


0.7 with  4 and  6


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.32it/s]


In [ ]:
test_levels('db26')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.06it/s]


0.6369565217391304 with  1 and  2


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.82it/s]


0.6652173913043478 with  1 and  3


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.40it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.25it/s]


0.6652173913043479 with  4 and  6


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.33it/s]


In [5]:
def p(n):
    return (n*chance+2)/(n+4)
def conf(n,p,alpha):
    return np.sqrt((p*(1-p))/(n+4))*norm.ppf(1-(alpha/2))
def genConf(n, alpha):
    res = np.zeros((n))
    for i in range(0,n):
        res[i] = p(i) + conf(i, p(i), alpha)
    return res[-1]*100

In [6]:
def test_levels(db, classifier='lda', selector='mrmr'):
    best_score = 0
    for lvl in range(6):
        for lvl1 in range(lvl,5):
            score = test_lvl_set(db, lvl+1, lvl1+2, classifier, selector)
            if score > best_score:
                best_score = score
                print(best_score, 'with ', lvl+1, 'and ', lvl1+2)
                
def test_lvl_set(db, lvl, lvl1, classifier, selector):
    score = []
    
    detail_coeffs, aprox_coeffs = get_coeffs(db, lvl, lvl1) 
    for train_rep in range(10):  
        cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=randint(19,48)+train_rep)        
        cv_split = cv.split(detail_coeffs[lvl], y)
        
        mrmr_features = None    
        median_score = []
        for train_idx, test_idx in cv_split:
            y_train, y_test = y[train_idx], y[test_idx]  

            x_train, x_test = get_feature_vector(detail_coeffs, aprox_coeffs, lvl, lvl1, train_idx, test_idx)

            if classifier == 'svm':
                estimator = svm_rbf
            elif classifier == 'lda':
                estimator = lda
            elif classifier == 'rf':
                estimator = rf
            elif classifier == 'knn':
                estimator = knn
                
            if selector == 'rfe':
                if rfe_features is None:
                    rfe = RFE(rf, n_features_to_select=10)
                    rfe.fit(x_train, y_train)
                    rfe_features = rfe.support_
                x_train =  x_train[:,rfe_features]
                x_test = x_test[:,rfe_features]
            elif selector == 'mrmr':
                if mrmr_features is None:            
                    x_pd = pd.DataFrame(x_train)
                    mrmr_features = mrmr_classif(X=x_pd, y=y_train, K=10)
                all_f = np.linspace(0,x_train.shape[1],x_train.shape[1]).astype(int)
                selected = [True if i in mrmr_features else False for i in all_f]
                
                x_train =  x_train[:,mrmr_features]
                x_test = x_test[:,mrmr_features]
            else:
                if relief_fit == False:
                    relief.fit(x_train, y_train)
                    relief_fit = True
                x_train = relief.transform(x_train.copy())
                x_test = relief.transform(x_test.copy())

            estimator.fit(x_train, y_train)
            median_score.append(np.median(estimator.score(x_test, y_test)))

        score.append(np.median(median_score))
    return np.mean(score)
def get_coeffs(db, lvl, lvl1): 
    x_aprox_coeff = train_data
          
    detail_coeffs = []
    aprox_coeffs = []
    for dwt_lvl in range(lvl1+1):
        x_det_coeff = np.apply_along_axis(dwt_det_coeff, 2, x_aprox_coeff, db=db)
        x_aprox_coeff = np.apply_along_axis(dwt_aprox_coeff, 2, x_aprox_coeff, db=db)

        detail_coeffs.append(scaler.fit_transform(x_det_coeff.copy(),y))
        aprox_coeffs.append(scaler.fit_transform(x_aprox_coeff.copy(),y))
    
    return detail_coeffs, aprox_coeffs

def get_feature_vector(detail_coeffs, aprox_coeffs, lvl, lvl1, train_idx, test_idx):
    x_train = []
    
    x_train.append(np.apply_along_axis(np.std, 2, detail_coeffs[lvl][train_idx]))
    x_train.append(np.apply_along_axis(np.max, 2, detail_coeffs[lvl][train_idx]))
    x_train.append(np.apply_along_axis(np.min, 2, detail_coeffs[lvl][train_idx]))
    x_train.append(np.apply_along_axis(rms, 2, detail_coeffs[lvl][train_idx]))
    x_train.append(np.apply_along_axis(slope, 2, detail_coeffs[lvl][train_idx]))
    x_train.append(np.apply_along_axis(skew, 2, detail_coeffs[lvl][train_idx]))
    x_train.append(np.apply_along_axis(np.average, 2, detail_coeffs[lvl][train_idx]**2))
    x_train.append(np.apply_along_axis(mean_abs_diff, 2, detail_coeffs[lvl][train_idx]))
    x_train.append(np.apply_along_axis(temp_centroid, 2, detail_coeffs[lvl][train_idx]))
    x_train.append(np.apply_along_axis(energy, 2, detail_coeffs[lvl][train_idx]))
    x_train.append(np.apply_along_axis(kurtosis, 2, detail_coeffs[lvl][train_idx]))
    x_train.append(np.apply_along_axis(calc_centroid, 2, detail_coeffs[lvl][train_idx]))

    x_train.append(np.apply_along_axis(np.std, 2, detail_coeffs[lvl1][train_idx]))
    x_train.append(np.apply_along_axis(np.max, 2, detail_coeffs[lvl1][train_idx]))
    x_train.append(np.apply_along_axis(np.min, 2, detail_coeffs[lvl1][train_idx]))
    x_train.append(np.apply_along_axis(rms, 2, detail_coeffs[lvl1][train_idx]))
    x_train.append(np.apply_along_axis(slope, 2, detail_coeffs[lvl1][train_idx]))
    x_train.append(np.apply_along_axis(skew, 2, detail_coeffs[lvl1][train_idx]))
    x_train.append(np.apply_along_axis(np.average, 2, detail_coeffs[lvl1][train_idx]**2))
    x_train.append(np.apply_along_axis(mean_abs_diff, 2, detail_coeffs[lvl1][train_idx]))
    x_train.append(np.apply_along_axis(temp_centroid, 2, detail_coeffs[lvl1][train_idx]))
    x_train.append(np.apply_along_axis(energy, 2, detail_coeffs[lvl1][train_idx]))
    x_train.append(np.apply_along_axis(kurtosis, 2, detail_coeffs[lvl1][train_idx]))
    x_train.append(np.apply_along_axis(calc_centroid, 2, detail_coeffs[lvl1][train_idx]))

    x_train.append(np.apply_along_axis(np.std, 2, aprox_coeffs[-1][train_idx]))
    x_train.append(np.apply_along_axis(np.max, 2, aprox_coeffs[-1][train_idx]))
    x_train.append(np.apply_along_axis(np.min, 2, aprox_coeffs[-1][train_idx]))
    x_train.append(np.apply_along_axis(rms, 2, aprox_coeffs[-1][train_idx]))
    x_train.append(np.apply_along_axis(slope, 2, aprox_coeffs[-1][train_idx]))
    x_train.append(np.apply_along_axis(skew, 2, aprox_coeffs[-1][train_idx]))
    x_train.append(np.apply_along_axis(np.average, 2, aprox_coeffs[-1][train_idx]**2))
    x_train.append(np.apply_along_axis(mean_abs_diff, 2, aprox_coeffs[-1][train_idx]))
    x_train.append(np.apply_along_axis(temp_centroid, 2, aprox_coeffs[-1][train_idx]))
    x_train.append(np.apply_along_axis(energy, 2, aprox_coeffs[-1][train_idx]))
    x_train.append(np.apply_along_axis(kurtosis, 2, aprox_coeffs[-1][train_idx]))
    x_train.append(np.apply_along_axis(calc_centroid, 2, aprox_coeffs[-1][train_idx]))

    x_train = np.concatenate(x_train, axis=1)

    x_test = []

    x_test.append(np.apply_along_axis(np.std, 2, detail_coeffs[lvl][test_idx]))
    x_test.append(np.apply_along_axis(np.max, 2, detail_coeffs[lvl][test_idx]))
    x_test.append(np.apply_along_axis(np.min, 2, detail_coeffs[lvl][test_idx]))
    x_test.append(np.apply_along_axis(rms, 2, detail_coeffs[lvl][test_idx]))
    x_test.append(np.apply_along_axis(slope, 2, detail_coeffs[lvl][test_idx]))
    x_test.append(np.apply_along_axis(skew, 2, detail_coeffs[lvl][test_idx]))
    x_test.append(np.apply_along_axis(np.average, 2, detail_coeffs[lvl][test_idx]**2))
    x_test.append(np.apply_along_axis(mean_abs_diff, 2, detail_coeffs[lvl][test_idx]))
    x_test.append(np.apply_along_axis(temp_centroid, 2, detail_coeffs[lvl][test_idx]))
    x_test.append(np.apply_along_axis(energy, 2, detail_coeffs[lvl][test_idx]))
    x_test.append(np.apply_along_axis(kurtosis, 2, detail_coeffs[lvl][test_idx]))
    x_test.append(np.apply_along_axis(calc_centroid, 2, detail_coeffs[lvl][test_idx]))

    x_test.append(np.apply_along_axis(np.std, 2, detail_coeffs[lvl1][test_idx]))
    x_test.append(np.apply_along_axis(np.max, 2, detail_coeffs[lvl1][test_idx]))
    x_test.append(np.apply_along_axis(np.min, 2, detail_coeffs[lvl1][test_idx]))
    x_test.append(np.apply_along_axis(rms, 2, detail_coeffs[lvl1][test_idx]))
    x_test.append(np.apply_along_axis(slope, 2, detail_coeffs[lvl1][test_idx]))
    x_test.append(np.apply_along_axis(skew, 2, detail_coeffs[lvl1][test_idx]))
    x_test.append(np.apply_along_axis(np.average, 2, detail_coeffs[lvl1][test_idx]**2))
    x_test.append(np.apply_along_axis(mean_abs_diff, 2, detail_coeffs[lvl1][test_idx]))
    x_test.append(np.apply_along_axis(temp_centroid, 2, detail_coeffs[lvl1][test_idx]))
    x_test.append(np.apply_along_axis(energy, 2, detail_coeffs[lvl1][test_idx]))
    x_test.append(np.apply_along_axis(kurtosis, 2, detail_coeffs[lvl1][test_idx]))
    x_test.append(np.apply_along_axis(calc_centroid, 2, detail_coeffs[lvl1][test_idx]))

    x_test.append(np.apply_along_axis(np.std, 2, aprox_coeffs[-1][test_idx]))
    x_test.append(np.apply_along_axis(np.max, 2, aprox_coeffs[-1][test_idx]))
    x_test.append(np.apply_along_axis(np.min, 2, aprox_coeffs[-1][test_idx]))
    x_test.append(np.apply_along_axis(rms, 2, aprox_coeffs[-1][test_idx]))
    x_test.append(np.apply_along_axis(slope, 2, aprox_coeffs[-1][test_idx]))
    x_test.append(np.apply_along_axis(skew, 2, aprox_coeffs[-1][test_idx]))
    x_test.append(np.apply_along_axis(np.average, 2, aprox_coeffs[-1][test_idx]**2))
    x_test.append(np.apply_along_axis(mean_abs_diff, 2, aprox_coeffs[-1][test_idx]))
    x_test.append(np.apply_along_axis(temp_centroid, 2, aprox_coeffs[-1][test_idx]))
    x_test.append(np.apply_along_axis(energy, 2, aprox_coeffs[-1][test_idx]))
    x_test.append(np.apply_along_axis(kurtosis, 2, aprox_coeffs[-1][test_idx]))
    x_test.append(np.apply_along_axis(calc_centroid, 2, aprox_coeffs[-1][test_idx]))

    x_test = np.concatenate(x_test, axis=1)

    return x_train, x_test


In [7]:
def test_extra_lvl(db, lvl, lvl1, classifier):
    pca_score = []
    rfe_score = []
    mrmr_score = []
    rf_score = []
    
    estimator = None
    rfe_features = None
    relief_fit = False
    
    detail_coeffs, aprox_coeffs = get_coeffs(db, lvl, lvl1) 
    for train_rep in range(10):  
        cv = StratifiedKFold(n_splits=3,shuffle=True, random_state=randint(15,35) + train_rep)        
        cv_split = cv.split(train_data, y)

        pca_median_score = []
        rfe_median_score = []
        mrmr_median_score = []
        rf_median_score = []
        
        mrmr_features = None
        for train_idx, test_idx in cv_split:
            y_train, y_test = y[train_idx], y[test_idx]

            x_train, x_test = get_feature_vector(detail_coeffs, aprox_coeffs, lvl, lvl1, train_idx, test_idx)
            
            if classifier == 'svm':
                estimator = svm_rbf
            elif classifier == 'lda':
                estimator = lda
            elif classifier == 'rf':
                estimator = rf
            elif classifier == 'knn':
                estimator = knn
                
            #pca       
            x_train_pca = pca.fit_transform(x_train.copy())
            x_test_pca = pca.transform(x_test.copy())  
            estimator.fit(x_train_pca, y_train)
            pca_median_score.append(np.median(estimator.score(x_test_pca, y_test)))

            # RFE
            if rfe_features is None:
                rfe = RFE(rf, n_features_to_select=10)
                rfe.fit(x_train, y_train)
                rfe_features = rfe.support_
            estimator.fit(x_train[:,rfe_features], y_train)
            rfe_median_score.append(np.median(estimator.score(x_test[:,rfe_features], y_test)))
            
            #mrmr
            if mrmr_features is None:            
                x_pd = pd.DataFrame(x_train)
                mrmr_features = mrmr_classif(X=x_pd, y=y_train, K=10)
            all_f = np.linspace(0,x_train.shape[1],x_train.shape[1]).astype(int)
            selected = [True if i in mrmr_features else False for i in all_f]
           
            estimator.fit(x_train[:,mrmr_features], y_train)
            mrmr_median_score.append(np.median(estimator.score(x_test[:,mrmr_features], y_test)))
                
            #ReliefF
            if relief_fit == False:
                relief.fit(x_train, y_train)
                relief_fit = True
            x_train_rf = relief.transform(x_train.copy())
            x_test_rf = relief.transform(x_test.copy())
            estimator.fit(x_train_rf, y_train)
            rf_median_score.append(np.median(estimator.score(x_test_rf, y_test)))
                
        pca_score.append(np.median(pca_median_score))
        rfe_score.append(np.median(rfe_median_score))
        mrmr_score.append(np.median(mrmr_median_score))
        rf_score.append(np.median(rf_median_score))

    print('pca',np.mean(pca_score))
    print('rfe',np.mean(rfe_score))
    print('mrmr',np.mean(mrmr_score))
    print('rf',np.mean(rf_score))

In [7]:
def test_dwt(db,lvl, lvl2=0):
    components = [.85,.87,.89,.91,.93,.95,.97,.99]
    
    best_score = 0
    for cmp in components:
        pca = PCA(n_components=cmp)        
        
        pca_score = []
        for train_rep in range(10):  
            cv = StratifiedKFold(n_splits=3,shuffle=True, random_state=25 + train_rep)        
            cv_split = cv.split(train_data, y)

            median_score = []
            for train_idx, test_idx in cv_split:
                y_train, y_test = y[train_idx], y[test_idx]
                x_aprox_coeff = train_data[train_idx]
                test_aprox_coeff = train_data[test_idx]

                detail_coeffs = []
                aprox_coeffs = []
                test_detail_coeffs = []
                test_aprox_coeffs = []
                dwt_csp = []
                test_dwt_csp = []
                total_energy = []
                test_total_energy = []

                for dwt_lvl in range(lvl+1):
                    x_det_coeff = np.apply_along_axis(dwt_det_coeff, 2, x_aprox_coeff, db=db)
                    x_aprox_coeff = np.apply_along_axis(dwt_aprox_coeff, 2, x_aprox_coeff, db=db)
                    test_det_coeff = np.apply_along_axis(dwt_det_coeff, 2, test_aprox_coeff, db=db)
                    test_aprox_coeff = np.apply_along_axis(dwt_aprox_coeff, 2, test_aprox_coeff, db=db)

                    detail_coeffs.append(scaler.fit_transform(x_det_coeff.copy(),y_train))
                    test_detail_coeffs.append(scaler.transform(test_det_coeff.copy()))
                    aprox_coeffs.append(scaler.fit_transform(x_aprox_coeff.copy(),y_train))
                    test_aprox_coeffs.append(scaler.transform(test_aprox_coeff.copy()))

                dwt_csp = detail_coeffs
                test_dwt_csp = test_detail_coeffs

                x_train = []

                x_train.append(np.apply_along_axis(np.std, 2, dwt_csp[lvl]))
                x_train.append(np.apply_along_axis(np.max, 2, dwt_csp[lvl]))
                x_train.append(np.apply_along_axis(np.min, 2, dwt_csp[lvl]))
    #             x_train.append(np.apply_along_axis(interq_range, 2, dwt_csp[lvl]))

                if lvl2 > 0:
                    x_train.append(np.apply_along_axis(np.max, 2, aprox_coeffs[lvl2]))
                    x_train.append(np.apply_along_axis(np.min, 2, aprox_coeffs[lvl2]))
                    x_train.append(np.apply_along_axis(np.std, 2, aprox_coeffs[lvl2]))
                    x_train.append(np.apply_along_axis(np.mean, 2, aprox_coeffs[lvl2]))

                x_train = np.concatenate(x_train, axis=1)

                x_test = []

                x_test.append(np.apply_along_axis(np.std, 2, test_dwt_csp[lvl]))
                x_test.append(np.apply_along_axis(np.max, 2, test_dwt_csp[lvl]))
                x_test.append(np.apply_along_axis(np.min, 2, test_dwt_csp[lvl]))
    #             x_test.append(np.apply_along_axis(interq_range, 2, test_dwt_csp[lvl]))

                if lvl2 > 0:
                    x_test.append(np.apply_along_axis(np.max, 2, test_aprox_coeffs[lvl2]))
                    x_test.append(np.apply_along_axis(np.min, 2, test_aprox_coeffs[lvl2]))
                    x_test.append(np.apply_along_axis(np.std, 2, test_aprox_coeffs[lvl2]))
                    x_test.append(np.apply_along_axis(np.mean, 2, test_aprox_coeffs[lvl2]))

                x_test = np.concatenate(x_test, axis=1)

                x_train = pca.fit_transform(x_train)
                x_test = pca.transform(x_test)        

                lda.fit(x_train, y_train)
                median_score.append(np.median(lda.score(x_test,y_test)))

            pca_score.append(np.median(median_score))

        if np.mean(pca_score) > best_score:
            best_score = np.mean(pca_score)
            print(best_score, ' ', cmp, ' pca')

In [201]:
def dwt_psd(db, lvl, lvl2=0, estimator='cov-lwf'):
    est_class, est_param = estimator.split('-')
    if est_class == "ker":
        psd = Kernels(metric=est_param)
    else:
        psd = Covariances(estimator=est_param)

        
    csp = CSP(n_components=4, reg=None, rank='info', transform_into='csp_space')
    
    score = []
    for train_rep in range(10):  
        cv = StratifiedKFold(n_splits=3,shuffle=True, random_state=25 + train_rep)        
        cv_split = cv.split(train_data, y)
        
        median_score = []
        for train_idx, test_idx in cv_split:
            y_train, y_test = y[train_idx], y[test_idx]
            x_aprox_coeff = train_data[train_idx]
            test_aprox_coeff = train_data[test_idx]
            
            detail_coeffs = []
            aprox_coeffs = []
            test_detail_coeffs = []
            test_aprox_coeffs = []
            dwt_csp = []
            test_dwt_csp = []
            total_energy = []
            test_total_energy = []

            for dwt_lvl in range(7):
                x_det_coeff = np.apply_along_axis(dwt_det_coeff, 2, x_aprox_coeff, db=db)
                x_aprox_coeff = np.apply_along_axis(dwt_aprox_coeff, 2, x_aprox_coeff, db=db)
                test_det_coeff = np.apply_along_axis(dwt_det_coeff, 2, test_aprox_coeff, db=db)
                test_aprox_coeff = np.apply_along_axis(dwt_aprox_coeff, 2, test_aprox_coeff, db=db)

#                 total_energy.append(np.apply_along_axis(np.sum, 2, scaler.transform(x_det_coeff.copy()**2)))
#                 test_total_energy.append(np.apply_along_axis(np.sum, 2, scaler.transform(test_det_coeff.copy()**2)))
                detail_coeffs.append(scaler.fit_transform(x_det_coeff.copy(),y_train))
                test_detail_coeffs.append(scaler.transform(test_det_coeff.copy()))
                aprox_coeffs.append(scaler.fit_transform(x_aprox_coeff.copy(),y_train))
                test_aprox_coeffs.append(scaler.transform(test_aprox_coeff.copy()))
                
#                 csp.fit(x_det_coeff, y_train)
#                 dwt_csp.append(csp.transform(x_det_coeff.copy()))
#                 test_dwt_csp.append(csp.transform(test_det_coeff.copy()))

            total_energy = np.apply_along_axis(np.sum, 0, total_energy)
            test_total_energy = np.apply_along_axis(np.sum, 0, test_total_energy)
   
            dwt_csp = detail_coeffs
            test_dwt_csp = test_detail_coeffs

            x_train = []
            
            x_train.append(np.apply_along_axis(np.std, 2, dwt_csp[lvl]))
            x_train.append(np.apply_along_axis(np.max, 2, dwt_csp[lvl]))
            x_train.append(np.apply_along_axis(np.min, 2, dwt_csp[lvl]))


            
            x_train = np.concatenate(x_train, axis=1)
            
#             epochs, channels, time = dwt_csp[lvl].shape 
#             train_aux = np.zeros((epochs, channels, time+3))
#             train_aux[:,:,:time] = dwt_csp[lvl]
            
#             for ft in range(len(x_train)):
#                 train_aux[:,:,time+ft] = x_train[ft]
#             x_train = train_aux
            
            x_test = []

            x_test.append(np.apply_along_axis(np.std, 2, test_dwt_csp[lvl]))
            x_test.append(np.apply_along_axis(np.max, 2, test_dwt_csp[lvl]))
            x_test.append(np.apply_along_axis(np.min, 2, test_dwt_csp[lvl]))

            x_test = np.concatenate(x_test, axis=1)
            
#             if lvl2 > 0:
#                 x_test.append(np.apply_along_axis(np.max, 2, test_dwt_csp[lvl2]))
#                 x_test.append(np.apply_along_axis(np.min, 2, test_dwt_csp[lvl2]))
            
#             epochs, channels, time = test_dwt_csp[lvl].shape 
            
#             test_aux = np.zeros((epochs, channels, time+3))
#             test_aux[:,:,:time] = test_dwt_csp[lvl]
            
#             for ft in range(len(x_test)):
#                 test_aux[:,:,time+ft] = x_test[ft]
#             x_test = test_aux
            
            psd_train = psd.fit_transform(dwt_csp[lvl], y_train)
            psd_test = psd.transform(test_dwt_csp[lvl])
            psd_train = tangent_space.fit_transform(psd_train)
            psd_test = tangent_space.transform(psd_test)
            
            x_train = np.concatenate([x_train,psd_train], axis=1)
            x_test = np.concatenate([x_test,psd_test], axis=1)
            
            x_train = pca.fit_transform(x_train)
            x_test = pca.transform(x_test)        
                        
            lda.fit(x_train, y_train)
            median_score.append(np.median(lda.score(x_test,y_test)))
            
        score.append(np.median(median_score))
    
    print(np.mean(score))
        

In [138]:
def rwe(x, total_energy):
    epochs, channels, time = x.shape
    rwe = np.zeros((epochs, channels))
    
    i = 0
    for epoch in x:
        j = 0
        for ch in epoch:
            rwe[i,j] = np.sum(epoch**2)/total_energy[i,j]
            j += 1
        i += 1
        
    return rwe
        